In [6]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


In [7]:
df = pd.read_csv("D:\Retail_store2.csv")

In [8]:
df.head()

,Date,Store ID,Product ID,Category,Region,Inventory Level,Units Sold,Units Ordered,Demand Forecast,Price,Discount,Weather Condition,Holiday/Promotion,Competitor Pricing,Seasonality
0,2022-01-01,S001,P0001,Groceries,North,231,127,55,135.47,33.50,20,Rainy,0,29.69,Autumn
1,2022-01-01,S001,P0002,Toys,South,204,150,66,144.04,63.01,20,Sunny,0,66.16,Autumn
2,2022-01-01,S001,P0003,Toys,West,102,65,51,74.02,27.99,10,Sunny,1,31.32,Summer
3,2022-01-01,S001,P0004,Toys,North,469,61,164,62.18,32.72,10,Cloudy,1,34.74,Autumn
4,2022-01-01,S001,P0005,Electronics,East,166,14,135,9.26,73.64,0,Sunny,0,68.95,Summer


In [9]:
df.shape

(73100, 15)

In [10]:
# convert  Data to datetime
df['Date'] = pd.to_datetime(df['Date'])

In [11]:
# Sorting is CRITICAL for lag features to work correctly
df = df.sort_values(by=['Store ID','Product ID','Date'])

In [12]:
# Helper function to print results clearly
def print_step_result(step_name, df, cols_to_show):
    print(f"\n--- {step_name} ---")
    valid_cols = [c for c in cols_to_show if c in df.columns]
    print(df[valid_cols].head(3).to_markdown(index=False, numalign="left", stralign="left"))

# 1) TIME-BASED FEATURES

In [13]:
df['Day'] = df['Date'].dt.day
df['month'] = df['Date'].dt.month
df['year'] = df['Date'].dt.year
df['day_of_week'] = df['Date'].dt.dayofweek
df['is_weeked'] = df['day_of_week'].apply(lambda x:1 if x >= 5 else 0)
print_step_result("Step 1: Time-Based Features", df, ['Date', 'day', 'month', 'year', 'day_of_week', 'is_weekend'])


--- Step 1: Time-Based Features ---
| Date                | month   | year   | day_of_week   |
|:--------------------|:--------|:-------|:--------------|
| 2022-01-01 00:00:00 | 1       | 2022   | 5             |
| 2022-01-02 00:00:00 | 1       | 2022   | 6             |
| 2022-01-03 00:00:00 | 1       | 2022   | 0             |


# 2) PRICE-BASED FEATURES

In [14]:
# Lag Prices (Shifted by Store/Product group)
df['price_lag_1'] = df.groupby(['Store ID', 'Product ID'])['Price'].shift(1)
df['price_lag_7'] = df.groupby(['Store ID', 'Product ID'])['Price'].shift(7)

# Price Change Percentage
df['price_change_pct'] = df.groupby(['Store ID', 'Product ID'])['Price'].pct_change()

# Discount Percentage
# Assuming 'Discount' is an integer percentage (e.g., 20 for 20%)
df['discount_pct'] = df['Discount'] / 100.0
print_step_result("Step 2: Price-Based Features", df, ['Date', 'Price', 'price_lag_1', 'price_change_pct', 'discount_pct'])


--- Step 2: Price-Based Features ---
| Date                | Price   | price_lag_1   | price_change_pct   | discount_pct   |
|:--------------------|:--------|:--------------|:-------------------|:---------------|
| 2022-01-01 00:00:00 | 33.5    | nan           | nan                | 0.2            |
| 2022-01-02 00:00:00 | 27.95   | 33.5          | -0.165672          | 0.1            |
| 2022-01-03 00:00:00 | 62.7    | 27.95         | 1.24329            | 0.2            |


# 3) DEMAND FEATURES (Lags and Rolling)

In [15]:
# Lags
df['sales_lag_1'] = df.groupby(['Store ID', 'Product ID'])['Units Sold'].shift(1)
df['sales_lag_7'] = df.groupby(['Store ID', 'Product ID'])['Units Sold'].shift(7)
df['sales_lag_30'] = df.groupby(['Store ID', 'Product ID'])['Units Sold'].shift(30)

# Rolling Means
# shift(1) is used to prevent data leakage (using today's sales to predict today)
grouped_sales = df.groupby(['Store ID', 'Product ID'])['Units Sold'].shift(1)

df['rolling_mean_7'] = grouped_sales.rolling(window=7, min_periods=1).mean().reset_index(0, drop=True)
df['rolling_mean_30'] = grouped_sales.rolling(window=30, min_periods=1).mean().reset_index(0, drop=True)

# Demand Volatility (Standard Deviation)
df['demand_volatility'] = grouped_sales.rolling(window=7, min_periods=1).std().reset_index(0, drop=True)
print_step_result("Step 3: Demand Features", df, ['Date', 'Units Sold', 'sales_lag_1', 'rolling_mean_7', 'demand_volatility'])


--- Step 3: Demand Features ---
| Date                | Units Sold   | sales_lag_1   | rolling_mean_7   | demand_volatility   |
|:--------------------|:-------------|:--------------|:-----------------|:--------------------|
| 2022-01-01 00:00:00 | 127          | nan           | nan              | nan                 |
| 2022-01-02 00:00:00 | 81           | 127           | 133.714          | 83.3561             |
| 2022-01-03 00:00:00 | 5            | 81            | 129              | 129.399             |


# 4. PRICE ELASTICITY

In [16]:
epsilon = 1e-6 # Small number to avoid division by zero
df['pct_change_demand'] = df.groupby(['Store ID', 'Product ID'])['Units Sold'].pct_change()

# Elasticity = % Change in Demand / % Change in Price
df['price_elasticity'] = df['pct_change_demand'] / (df['price_change_pct'] + epsilon)

# Clean infinite values
df['price_elasticity'] = df['price_elasticity'].replace([np.inf, -np.inf], np.nan)

# Classification
def classify_elasticity(x):
    if pd.isna(x): return 'Unknown'
    if x < -1: return 'High'   # Elastic (Price sensitive)
    elif -1 <= x < 0: return 'Medium' # Inelastic
    else: return 'Low'         # Positive/Zero (Unusual)

df['elasticity_type'] = df['price_elasticity'].apply(classify_elasticity)
print_step_result("Step 4: Price Elasticity", df, ['Date', 'Price', 'Units Sold', 'price_elasticity', 'elasticity_type'])


--- Step 4: Price Elasticity ---
| Date                | Price   | Units Sold   | price_elasticity   | elasticity_type   |
|:--------------------|:--------|:-------------|:-------------------|:------------------|
| 2022-01-01 00:00:00 | 33.5    | 127          | nan                | Unknown           |
| 2022-01-02 00:00:00 | 27.95   | 81           | 2.18629            | Low               |
| 2022-01-03 00:00:00 | 62.7    | 5            | -0.754667          | Medium            |


# 5. COMPETITOR FEATURES

In [17]:
if 'Competitor Pricing' in df.columns:
    df['competitor_diff'] = df['Price'] - df['Competitor Pricing']
    df['competitor_index'] = df['Price'] / (df['Competitor Pricing'] + epsilon)
    df['is_competitor_cheaper'] = (df['Competitor Pricing'] < df['Price']).astype(int)
    print_step_result("Step 5: Competitor Features", df, ['Date', 'Price', 'Competitor Pricing', 'competitor_diff', 'is_competitor_cheaper'])


--- Step 5: Competitor Features ---
| Date                | Price   | Competitor Pricing   | competitor_diff   | is_competitor_cheaper   |
|:--------------------|:--------|:---------------------|:------------------|:------------------------|
| 2022-01-01 00:00:00 | 33.5    | 29.69                | 3.81              | 1                       |
| 2022-01-02 00:00:00 | 27.95   | 30.89                | -2.94             | 0                       |
| 2022-01-03 00:00:00 | 62.7    | 58.22                | 4.48              | 1                       |


# 6. INVENTORY FEATURES

In [18]:
# Inventory Ratio (Days of supply based on forecasted demand)
df['inventory_coverage_days'] = df['Inventory Level'] / (df['Demand Forecast'] + epsilon)

# Days until stockout (based on historical average sales)
avg_daily_sales = df.groupby(['Store ID', 'Product ID'])['Units Sold'].transform('mean')
df['days_until_stockout'] = df['Inventory Level'] / (avg_daily_sales + epsilon)

# Indicators
df['is_low_stock'] = (df['days_until_stockout'] < 3).astype(int)
df['is_overstock'] = (df['days_until_stockout'] > 30).astype(int)
print_step_result("Step 6: Inventory Features", df, ['Date', 'Inventory Level', 'days_until_stockout', 'is_low_stock'])


--- Step 6: Inventory Features ---
| Date                | Inventory Level   | days_until_stockout   | is_low_stock   |
|:--------------------|:------------------|:----------------------|:---------------|
| 2022-01-01 00:00:00 | 231               | 1.68237               | 1              |
| 2022-01-02 00:00:00 | 116               | 0.844826              | 1              |
| 2022-01-03 00:00:00 | 154               | 1.12158               | 1              |


# 7. PROFIT FEATURES

In [19]:
# NOTE: Generating synthetic cost because 'Cost' column was missing.
# REPLACE this with: df['Cost'] if you have it.
df['estimated_cost'] = df['Price'] / 1.3  # Assuming ~30% markup
df['profit_per_unit'] = df['Price'] - df['estimated_cost']
df['profit_margin'] = (df['profit_per_unit'] / df['Price']) * 100
print_step_result("Step 7: Profit Features", df, ['Date', 'Price', 'estimated_cost', 'profit_margin'])


--- Step 7: Profit Features ---
| Date                | Price   | estimated_cost   | profit_margin   |
|:--------------------|:--------|:-----------------|:----------------|
| 2022-01-01 00:00:00 | 33.5    | 25.7692          | 23.0769         |
| 2022-01-02 00:00:00 | 27.95   | 21.5             | 23.0769         |
| 2022-01-03 00:00:00 | 62.7    | 48.2308          | 23.0769         |


# 8. INTERACTION FEATURES

In [20]:
df['interact_weekend_price'] = df['Holiday/Promotion'] * df['Price']
df['interact_inventory_price'] = df['Inventory Level'] * df['Price']
df['interact_month_discount'] = df['month'] * df['Discount']
print_step_result("Step 8: Interaction Features", df, ['Date', 'is_weekend', 'Price', 'interact_weekend_price'])


--- Step 8: Interaction Features ---
| Date                | Price   | interact_weekend_price   |
|:--------------------|:--------|:-------------------------|
| 2022-01-01 00:00:00 | 33.5    | 0                        |
| 2022-01-02 00:00:00 | 27.95   | 0                        |
| 2022-01-03 00:00:00 | 62.7    | 0                        |


# 9. CATEGORICAL ENCODING

In [21]:
# Label Encoding for high cardinality IDs
le = LabelEncoder()
df['Store_ID_Enc'] = le.fit_transform(df['Store ID'])
le_prod = LabelEncoder()
df['Product_ID_Enc'] = le_prod.fit_transform(df['Product ID'])

# One-Hot Encoding for low cardinality categories
categorical_cols = ['Category', 'Region', 'Weather Condition', 'Seasonality', 'elasticity_type']
# Ensure columns are string type before encoding
for c in categorical_cols:
    df[c] = df[c].astype(str)

ohe = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')
ohe_encoded = ohe.fit_transform(df[categorical_cols])

feature_names = ohe.get_feature_names_out(categorical_cols)
df_ohe = pd.DataFrame(ohe_encoded, columns=feature_names, index=df.index)

# Concatenate with original data and drop the old categorical columns
df = pd.concat([df, df_ohe], axis=1).drop(columns=categorical_cols)

print_step_result("Step 9: Categorical Encoding (Corrected)", df, ['Date', 'Store_ID_Enc', 'Product_ID_Enc'] + list(feature_names[:3]))


--- Step 9: Categorical Encoding (Corrected) ---
| Date                | Store_ID_Enc   | Product_ID_Enc   | Category_Electronics   | Category_Furniture   | Category_Groceries   |
|:--------------------|:---------------|:-----------------|:-----------------------|:---------------------|:---------------------|
| 2022-01-01 00:00:00 | 0              | 0                | 0                      | 0                    | 1                    |
| 2022-01-02 00:00:00 | 0              | 0                | 0                      | 0                    | 1                    |
| 2022-01-03 00:00:00 | 0              | 0                | 1                      | 0                    | 0                    |


# 10. FINAL CLEANING & SCALING

In [22]:

from sklearn.preprocessing import MinMaxScaler

# Drop rows with NaNs generated by Lags (first 30 days)
df.dropna(inplace=True)

# Remove Outliers (IQR Method)
Q1 = df['Units Sold'].quantile(0.25)
Q3 = df['Units Sold'].quantile(0.75)
IQR = Q3 - Q1
df = df[~((df['Units Sold'] < (Q1 - 1.5 * IQR)) | (df['Units Sold'] > (Q3 + 1.5 * IQR)))]

# Scale Numerical Features (MinMax)
scaler = MinMaxScaler()
cols_to_scale = [
    'Price', 'Inventory Level', 'Demand Forecast', 'Competitor Pricing',
    'rolling_mean_7', 'rolling_mean_30', 'demand_volatility',
    'inventory_coverage_days', 'days_until_stockout'
]
# Only scale columns that exist in the dataframe
cols_to_scale = [c for c in cols_to_scale if c in df.columns]

df[cols_to_scale] = scaler.fit_transform(df[cols_to_scale])

# Save Final File
print("Processing complete. Final shape:", df.shape)
df.to_csv('Retail_store_processed.csv', index=False)

print_step_result("Step 10: Final Cleaning & Scaling", df, ['Date', 'Price', 'Inventory Level'])


Processing complete. Final shape: (69097, 59)

--- Step 10: Final Cleaning & Scaling ---
| Date                | Price    | Inventory Level   |
|:--------------------|:---------|:------------------|
| 2022-01-31 00:00:00 | 0.675333 | 0.8               |
| 2022-02-01 00:00:00 | 0.272111 | 0.82              |
| 2022-02-02 00:00:00 | 0.472111 | 0.811111          |


In [23]:
print("final shape of the data:",df.shape)

final shape of the data: (69097, 59)


In [26]:
df.head


<bound method NDFrame.head of             Date Store ID Product ID  Inventory Level  Units Sold  \
3000  2022-01-31     S001      P0001         0.800000         200   
3100  2022-02-01     S001      P0001         0.820000         279   
3200  2022-02-02     S001      P0001         0.811111          38   
3300  2022-02-03     S001      P0001         0.655556          71   
3400  2022-02-04     S001      P0001         0.157778          25   
...          ...      ...        ...              ...         ...   
72699 2023-12-28     S005      P0020         0.328889          56   
72799 2023-12-29     S005      P0020         0.880000         268   
72899 2023-12-30     S005      P0020         0.446667         149   
72999 2023-12-31     S005      P0020         0.031111          40   
73099 2024-01-01     S005      P0020         0.148889           6   

       Units Ordered  Demand Forecast     Price  Discount  Holiday/Promotion  \
3000             152         0.478398  0.675333         0    

In [27]:
df.head()

,Date,Store ID,Product ID,Inventory Level,Units Sold,Units Ordered,Demand Forecast,Price,Discount,Holiday/Promotion,...,Region_West,Weather Condition_Rainy,Weather Condition_Snowy,Weather Condition_Sunny,Seasonality_Spring,Seasonality_Summer,Seasonality_Winter,elasticity_type_Low,elasticity_type_Medium,elasticity_type_Unknown
3000,2022-01-31,S001,P0001,0.800000,200,152,0.478398,0.675333,0,1,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
3100,2022-02-01,S001,P0001,0.820000,279,84,0.661421,0.272111,0,0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
3200,2022-02-02,S001,P0001,0.811111,38,149,0.135879,0.472111,5,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3300,2022-02-03,S001,P0001,0.655556,71,186,0.202377,0.196778,20,1,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3400,2022-02-04,S001,P0001,0.157778,25,25,0.102706,0.076000,15,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
